In [ ]:
import numpy as np
import os.path as op
import pandas as pd
import mne
import ast




In [ ]:
#dictionary of raw recordigns per patient
subjects = {"patient_X" : "file path"}

In [ ]:
#dictionary of fiducials text files 
fiudicals_files = {"patient_X" : "file path"}

In [ ]:
#funciton to read dictionary from a file

def read_dictionary_from_file(filepath,sf):
    data_dict = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 4:
                    key = parts[0]
                    array_values = parts[1:4]
                    try:
                        array = np.array([float(val) for val in array_values])
                        data_dict[key] = array * sf
                    except ValueError:
                        print(f"Warning: Could not convert array values to floats for key '{key}'. Skipping line.")
                else:
                    print(f"Warning: Insufficient columns in line: '{line.strip()}'. Skipping line.")

    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'.")
    except Exception as e:
        print(f"An unexpected error occured: {e}")

    return data_dict

In [ ]:
for subject, path in subjects.items():
    raw_fname = op.abspath( subjects[subject])
    raw = mne.io.read_raw_fif(raw_fname)
    info = raw.info


    fiducials = read_dictionary_from_file(fiudicals_files[subject], 0.001)
    print(fiducials)
    fid_viz = "estimated"
   

    data_path = op.abspath("path")
    subjects_dir = op.join(data_path, "new_patient_coregs")
    coreg = Coregistration(info, subject, subjects_dir, fiducials=fiducials)
    coreg.fit_fiducials(nasion_weight=1,verbose=True)
   
    #save files
    coreg_dir = op.join(data_path, "coregs")
    trans_fname = op.join(coreg_dir, f"{subject}-trans.fif")
    mne.write_trans(trans_fname, coreg.trans, overwrite = True)
    print(f"Saved transformation to: {trans_fname}")

    ## visualisation options to check if the alignment looks okay
    plot_kwargs = dict(
        subject=subject,
        subjects_dir=subjects_dir,
        surfaces="head",
        dig=True,
        eeg=[],
        meg="sensors",
        show_axes=True,
        coord_frame="meg",
    )

    fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
